In [1]:
# global libs
import torch as pt
from glob import glob
import os
# import sys

/home/dima/miniconda3/envs/ctpesto/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# project libs
from src.data_encoding import encode_res, all_resnames, selected_locations, encode_location
from config import config_data, config_runtime
from model import RNN
from src.dataset import fasta_to_vector, read_fasta

# Load model

In [4]:
# define 
nres = config_data['sequence_max_length']
device = pt.device("cpu")
n_letters = len(all_resnames)
n_categories = len(selected_locations)
learning_rate = config_runtime['learning_rate']
n_hidden = config_runtime['hidden_size']
n_layers = config_runtime['layers']
# create model
model = RNN(nres, n_hidden, n_layers, n_categories, device)
# reload model
model.load_state_dict(pt.load("model.pt", map_location=pt.device("cpu")))
# set model to inference
model = model.eval().to(device)

In [5]:
# data parameters

data_path = "../examples/"
sm = pt.nn.Softmax(dim=1)

In [6]:
pdb_filepaths = glob(os.path.join(data_path, "*.*"))

In [7]:
all_vec = []
all_names = []
for fasta_file in pdb_filepaths:
    a,b = read_fasta(fasta_file)
    
    v = fasta_to_vector(a, nres)
    
    if not(v is None):
        with pt.no_grad():
            results = sm(model(v))[0]
        print()
        print(f'===== {b: ^15} ====')
        j = results.detach().numpy()

        for il, l in enumerate(selected_locations):
            print(f'{l: <15} p={j[il]:>5.2f}')


=====   SAR1_YEAST    ====
membrane        p= 1.00
cytoplasm       p= 0.00
nucle           p= 0.00
vesicl          p= 0.00

=====   CCL20_MOUSE   ====
membrane        p= 1.00
cytoplasm       p= 0.00
nucle           p= 0.00
vesicl          p= 0.00

=====   GID4_MOUSE    ====
membrane        p= 1.00
cytoplasm       p= 0.00
nucle           p= 0.00
vesicl          p= 0.00

=====   HDAC3_DANRE   ====
membrane        p= 1.00
cytoplasm       p= 0.00
nucle           p= 0.00
vesicl          p= 0.00

=====    XCL1_RAT     ====
membrane        p= 0.81
cytoplasm       p= 0.12
nucle           p= 0.06
vesicl          p= 0.01

=====   ICP27_HHV1E   ====
membrane        p= 0.03
cytoplasm       p= 0.13
nucle           p= 0.83
vesicl          p= 0.01

=====   MCL1_MOUSE    ====
membrane        p= 0.55
cytoplasm       p= 0.28
nucle           p= 0.14
vesicl          p= 0.02

=====   FTSZ_BACSU    ====
membrane        p= 0.04
cytoplasm       p= 0.24
nucle           p= 0.70
vesicl          p= 0.02

=====  